# Dataset Source: OTTO – Multi-Objective Recommender System

github: https://github.com/otto-de/recsys-dataset

kaggle: https://www.kaggle.com/competitions/otto-recommender-system/overview

In [3]:
import pandas as pd

In [ ]:
pd.load_json(